В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [2]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 59.4 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [3]:
fasttext_model = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [5]:
def ten_closest(word, model):
    return model.most_similar(word, topn=10)
ten_closest('python', fasttext_model)

[('reticulated', 0.6916365623474121),
 ('spamalot', 0.6635736227035522),
 ('php', 0.6414496898651123),
 ('owl', 0.6301496028900146),
 ('mouse', 0.6275478601455688),
 ('reticulatus', 0.6274471282958984),
 ('perl', 0.6267575025558472),
 ('monkey', 0.6207211017608643),
 ('monty', 0.60793536901474),
 ('scripting', 0.6041731834411621)]

3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [7]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [8]:
model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

In [9]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [10]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [11]:
# Найдите слова, похожие на "духовка"
### ваш код здесь ###
for i in ['духовка', 'овощи']:
    try:
        similar = model.wv.most_similar(i, topn=5)
        print(f"Слова, похожие на {i}:")
        for word, score in similar:
            print(f"  {word}: {score:.4f}")
    except KeyError:
        print(f"Слово {i} не найдено в словаре")
# Найдите слова, похожие на "овощи"
### ваш код здесь ###

Слова, похожие на духовка:
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на овощи:
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [15]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [16]:
for i in ['варить', 'духовка', 'овощи']:
    try:
        similar = ft_model.wv.most_similar(i, topn=5)
        print(f"Слова, похожие на {i}:")
        for word, score in similar:
            print(f"  {word}: {score:.4f}")
    except KeyError:
        print(f"Слово {i} не найдено в словаре")

Слова, похожие на варить:
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
Слова, похожие на духовка:
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
Слова, похожие на овощи:
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [17]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('learning')
compare_models('neural')
compare_models('марковь')
compare_models('мисить')
compare_models('хлеп')


Сравнение для слова: 'learning'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'пирог']

Сравнение для слова: 'neural'
  Word2Vec: слово не найдено
  FastText: ['мука', 'травы']

Сравнение для слова: 'марковь'
  Word2Vec: слово не найдено
  FastText: ['морковь', 'овощи']

Сравнение для слова: 'мисить'
  Word2Vec: слово не найдено
  FastText: ['начинка', 'бекон']

Сравнение для слова: 'хлеп'
  Word2Vec: слово не найдено
  FastText: ['хлеб', 'здоровое']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [18]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [19]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [20]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [21]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [22]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [23]:
similar_docs = doc_model.dv.most_similar("doc_1", topn=2)
print("\nДокументы, похожие на doc_1:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")


Документы, похожие на doc_1:
  doc_0: 0.2735
    Текст: machine learning is interesting
  doc_3: 0.2031
    Текст: artificial intelligence is amazing


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [25]:
for size in [10, 50, 100]:
    ft_model = FastText(
        sentences=cooking_sentences,
        vector_size=size,
        window=3,
        min_count=1,
        workers=2
    )
    print(f'Размерность - {size}')
    for i in ['овощ', 'блюдо', 'готовка']:
        try:
            similar = ft_model.wv.most_similar(i, topn=5)
            print(f"Слова, похожие на {i}:")
            for word, score in similar:
                print(f"  {word}: {score:.4f}")
        except KeyError:
            print(f"Слово {i} не найдено в словаре")

Размерность - 10
Слова, похожие на овощ:
  десерт: 0.6679
  гриль: 0.6150
  питание: 0.5196
  масло: 0.5142
  смешивать: 0.3940
Слова, похожие на блюдо:
  дрожжи: 0.5394
  огурцы: 0.5157
  вино: 0.5117
  горшок: 0.4178
  барбекю: 0.4119
Слова, похожие на готовка:
  травы: 0.8046
  фольга: 0.7035
  чашка: 0.6939
  мука: 0.5771
  варить: 0.5587
Размерность - 50
Слова, похожие на овощ:
  овощи: 0.4671
  десерт: 0.2193
  бекон: 0.1490
  кофе: 0.1331
  ингредиенты: 0.1288
Слова, похожие на блюдо:
  месить: 0.2970
  жарить: 0.2658
  торт: 0.2308
  кипятить: 0.2223
  огурцы: 0.2067
Слова, похожие на готовка:
  готовить: 0.4577
  рыба: 0.4259
  месить: 0.3614
  вода: 0.3485
  взбивать: 0.3296
Размерность - 100
Слова, похожие на овощ:
  овощи: 0.5020
  огурцы: 0.2171
  варить: 0.2012
  жарить: 0.1977
  рыба: 0.1556
Слова, похожие на блюдо:
  салат: 0.2671
  варить: 0.2337
  готовить: 0.2017
  чашка: 0.1876
  торт: 0.1777
Слова, похожие на готовка:
  готовить: 0.4360
  духовка: 0.2949
  салат: 0